# 检测器模块

## 数据集处理（转化为CocoVID格式）

### 训练集

In [5]:
# convert the dataset to coco format
!python ./tools/convert_datasets/visdronemot/visdronemot2coco.py -d ./data/VisDrone_MOT_tiny/train

100%|█████████████████████████████████████████████| 2/2 [00:00<00:00,  9.24it/s]


### 测试集

In [7]:
# convert the dataset to coco format
!python ./tools/convert_datasets/visdronemot/visdronemot2coco.py -d ./data/VisDrone_MOT_tiny/test

100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  6.48it/s]


## 修改训练配置

In [8]:
import mmcv
from mmdet.apis import set_random_seed
cfg = mmcv.Config.fromfile('./configs/det/faster-rcnn_r50_fpn_4e_visdrone_mot.py')
cfg.data_root = 'data/VisDrone_MOT_tiny/'
cfg.data.test.ann_file = cfg.data.test.ann_file.replace('data/VisDrone_MOT/','data/VisDrone_MOT_tiny/')
cfg.data.train.ann_file = cfg.data.train.ann_file.replace('data/VisDrone_MOT/','data/VisDrone_MOT_tiny/')
cfg.data.val.ann_file = cfg.data.val.ann_file.replace('data/VisDrone_MOT/','data/VisDrone_MOT_tiny/')

cfg.data.test.img_prefix = cfg.data.test.img_prefix.replace('data/VisDrone_MOT/','data/VisDrone_MOT_tiny/')
cfg.data.train.img_prefix = cfg.data.train.img_prefix.replace('data/VisDrone_MOT/','data/VisDrone_MOT_tiny/')
cfg.data.val.img_prefix = cfg.data.val.img_prefix.replace('data/VisDrone_MOT/','data/VisDrone_MOT_tiny/')

cfg.work_dir = './tutorial_exps/detector'
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.device = "cuda"
# cfg.gpu_ids = range(1)
cfg.gpu_ids = [1]
print(f'Config:\n{cfg.pretty_text}')

Config:
model = dict(
    detector=dict(
        type='FasterRCNN',
        backbone=dict(
            type='ResNet',
            depth=50,
            num_stages=4,
            out_indices=(0, 1, 2, 3),
            frozen_stages=1,
            norm_cfg=dict(type='BN', requires_grad=True),
            norm_eval=True,
            style='pytorch',
            init_cfg=dict(
                type='Pretrained', checkpoint='torchvision://resnet50')),
        neck=dict(
            type='FPN',
            in_channels=[256, 512, 1024, 2048],
            out_channels=256,
            num_outs=5),
        rpn_head=dict(
            type='RPNHead',
            in_channels=256,
            feat_channels=256,
            anchor_generator=dict(
                type='AnchorGenerator',
                scales=[8],
                ratios=[0.5, 1.0, 2.0],
                strides=[4, 8, 16, 32, 64]),
            bbox_coder=dict(
                type='DeltaXYWHBBoxCoder',
                target_means=[0.0,

## 检测器训练&验证

In [9]:
import os.path as osp

from mmtrack.datasets import build_dataset
from mmdet.apis import train_detector as train_model
from mmdet.models import build_detector as build_model

mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
model = build_model(cfg.model.detector)
model.init_weights()
datasets = [build_dataset(cfg.data.train)]
model.CLASSES = datasets[0].CLASSES
train_model(model, datasets, cfg, validate=True)

2022-11-08 02:50:45,570 - mmcv - INFO - initialize FasterRCNN with init_cfg {'type': 'Pretrained', 'checkpoint': 'http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_2x_coco/faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8aa15.pth'}
2022-11-08 02:50:45,570 - mmcv - INFO - load model from: http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_2x_coco/faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8aa15.pth
2022-11-08 02:50:45,571 - mmcv - INFO - load checkpoint from http path: http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_2x_coco/faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8aa15.pth
2022-11-08 02:50:45,663 - mmcv - WARNING - The model and loaded state dict do not match exactly

size mismatch for roi_head.bbox_head.fc_cls.weight: copying a param with shape torch.Size([81, 1024]) from checkpoint, the shape in current model is torch.Size([2, 1024]).
size mism

loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


2022-11-08 02:51:04,189 - mmdet - INFO - Epoch [1][50/164]	lr: 9.902e-03, eta: 0:03:41, time: 0.366, data_time: 0.047, memory: 3428, loss_rpn_cls: 0.0421, loss_rpn_bbox: 0.0666, loss_cls: 0.3353, acc: 85.1836, loss_bbox: 0.2447, loss: 0.6887
2022-11-08 02:51:20,519 - mmdet - INFO - Epoch [1][100/164]	lr: 1.980e-02, eta: 0:03:12, time: 0.326, data_time: 0.005, memory: 3428, loss_rpn_cls: 0.0237, loss_rpn_bbox: 0.0653, loss_cls: 0.1906, acc: 91.7910, loss_bbox: 0.1352, loss: 0.4148
2022-11-08 02:51:37,020 - mmdet - INFO - Epoch [1][150/164]	lr: 2.000e-02, eta: 0:02:52, time: 0.330, data_time: 0.005, memory: 3428, loss_rpn_cls: 0.0257, loss_rpn_bbox: 0.0562, loss_cls: 0.1543, acc: 93.3398, loss_bbox: 0.1155, loss: 0.3517
2022-11-08 02:51:41,637 - mmdet - INFO - Saving checkpoint at 1 epochs


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 118/118, 13.0 task/s, elapsed: 9s, ETA:     0s

2022-11-08 02:51:51,464 - mmdet - INFO - Evaluating bbox...


...

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 118/118, 14.7 task/s, elapsed: 8s, ETA:     0s

2022-11-08 02:55:09,756 - mmdet - INFO - Evaluating bbox...


Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*


2022-11-08 02:55:10,492 - mmdet - INFO - 
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.123
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.290
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.085
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.014
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.162
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.161
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.195
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.195
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.195
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.088
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.216
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000

DONE (t=0.67s).
Accumulating evaluation results...
DONE (t=0.06s).


# ReID模块

## 数据集处理（转化为ReID数据集）

In [17]:
!rm -r ./data/VisDrone_MOT_tiny/train/reid 
!rm -r ./data/VisDrone_MOT_tiny/train/reid_meta
!rm -r ./data/VisDrone_MOT_tiny/test/reid 
!rm -r ./data/VisDrone_MOT_tiny/test/reid_meta

rm: cannot remove './data/VisDrone_MOT_tiny/test/reid': No such file or directory
rm: cannot remove './data/VisDrone_MOT_tiny/test/reid_meta': No such file or directory


In [18]:
!python ./tools/convert_datasets/visdronemot/visdronemot2reid.py -d ./data/VisDrone_MOT_tiny/train --min-per-person=4 --max-per-person=10

corp reid images...
100%|█████████████████████████████████████████████| 2/2 [00:04<00:00,  2.23s/it]
generate reid labels...


In [19]:
!python ./tools/convert_datasets/visdronemot/visdronemot2reid.py -d ./data/VisDrone_MOT_tiny/test --min-per-person=4 --max-per-person=10

corp reid images...
100%|█████████████████████████████████████████████| 1/1 [00:01<00:00,  1.47s/it]
generate reid labels...


## 修改训练配置

In [28]:
import mmcv
from mmdet.apis import set_random_seed
cfg = mmcv.Config.fromfile('./configs/reid/resnet50_b32x8_VisDroneMOT.py')
cfg.data_root = 'data/VisDrone_MOT_tiny/'
# cfg.data.train.ann_file = 'data/VisDrone_MOT_tiny/train/reid_meta/train.txt'
cfg.data.train.ann_file = cfg.data.train.ann_file.replace('data/VisDrone_MOT/','data/VisDrone_MOT_tiny/')
cfg.data.val.ann_file = cfg.data.val.ann_file.replace('data/VisDrone_MOT/','data/VisDrone_MOT_tiny/')
cfg.data.test.ann_file = cfg.data.test.ann_file.replace('data/VisDrone_MOT/','data/VisDrone_MOT_tiny/')

cfg.data.train.data_prefix = cfg.data.train.data_prefix.replace('data/VisDrone_MOT/','data/VisDrone_MOT_tiny/')
cfg.data.val.data_prefix = cfg.data.val.data_prefix.replace('data/VisDrone_MOT/','data/VisDrone_MOT_tiny/')
cfg.data.test.data_prefix = cfg.data.test.data_prefix.replace('data/VisDrone_MOT/','data/VisDrone_MOT_tiny/')

# learning policy
cfg.lr_config = dict(
    policy='step',
    warmup='linear',
    warmup_iters=200,
    warmup_ratio=1.0 / 200,
    step=[1])
cfg.total_epochs = 2

cfg.work_dir = './tutorial_exps/reid'
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.device = "cuda"
# cfg.gpu_ids = range(1)
cfg.gpu_ids = [0]
print(f'Config:\n{cfg.pretty_text}')

Config:
dataset_type = 'ReIDDataset'
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
train_pipeline = [
    dict(type='LoadMultiImagesFromFile', to_float32=True),
    dict(
        type='SeqResize',
        img_scale=(128, 256),
        share_params=False,
        keep_ratio=False,
        bbox_clip_border=False,
        override=False),
    dict(
        type='SeqRandomFlip',
        share_params=False,
        flip_ratio=0.5,
        direction='horizontal'),
    dict(
        type='SeqNormalize',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        to_rgb=True),
    dict(type='VideoCollect', keys=['img', 'gt_label']),
    dict(type='ReIDFormatBundle')
]
test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='Resize', img_scale=(128, 256), keep_ratio=False),
    dict(
        type='Normalize',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        to_rgb=True),

## ReID模块训练&验证

In [29]:
from mmtrack.datasets import build_dataset
from mmdet.apis import train_detector as train_model
from mmtrack.models import build_reid as build_model


model = build_model(cfg.model.reid)
model.init_weights()
datasets = [build_dataset(cfg.data.train)]
model.CLASSES = datasets[0].CLASSES

train_model(model, datasets, cfg, validate=True)

2022-11-08 03:12:05,701 - mmcv - INFO - initialize BaseReID with init_cfg {'type': 'Pretrained', 'checkpoint': 'https://download.openmmlab.com/mmclassification/v0/resnet/resnet50_batch256_imagenet_20200708-cfb998bf.pth'}
2022-11-08 03:12:05,702 - mmcv - INFO - load model from: https://download.openmmlab.com/mmclassification/v0/resnet/resnet50_batch256_imagenet_20200708-cfb998bf.pth
2022-11-08 03:12:05,702 - mmcv - INFO - load checkpoint from http path: https://download.openmmlab.com/mmclassification/v0/resnet/resnet50_batch256_imagenet_20200708-cfb998bf.pth
2022-11-08 03:12:05,747 - mmcv - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: head.fc.weight, head.fc.bias

missing keys in source state_dict: head.fcs.0.fc.weight, head.fcs.0.fc.bias, head.fcs.0.bn.weight, head.fcs.0.bn.bias, head.fcs.0.bn.running_mean, head.fcs.0.bn.running_var, head.fc_out.weight, head.fc_out.bias, head.bn.weight, head.bn.bias, head.bn.running_mean, head.bn.

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 294/294, 132.5 task/s, elapsed: 2s, ETA:     0s

2022-11-08 03:13:30,394 - mmdet - INFO - Epoch(val) [1][294]	mAP: 0.6870
2022-11-08 03:13:39,146 - mmdet - INFO - Epoch [2][50/604]	lr: 1.000e-02, eta: 0:01:15, time: 0.174, data_time: 0.045, memory: 2587, triplet_loss: 0.2025, ce_loss: 1.8454, top-1: 41.8750, loss: 2.0479
2022-11-08 03:13:45,746 - mmdet - INFO - Epoch [2][100/604]	lr: 1.000e-02, eta: 0:01:08, time: 0.132, data_time: 0.003, memory: 2587, triplet_loss: 0.1795, ce_loss: 1.4846, top-1: 54.3125, loss: 1.6641
2022-11-08 03:13:52,342 - mmdet - INFO - Epoch [2][150/604]	lr: 1.000e-02, eta: 0:01:01, time: 0.132, data_time: 0.003, memory: 2587, triplet_loss: 0.2270, ce_loss: 1.4436, top-1: 50.8750, loss: 1.6706
2022-11-08 03:13:58,934 - mmdet - INFO - Epoch [2][200/604]	lr: 1.000e-02, eta: 0:00:54, time: 0.132, data_time: 0.003, memory: 2587, triplet_loss: 0.1133, ce_loss: 1.3199, top-1: 56.5625, loss: 1.4332
2022-11-08 03:14:05,525 - mmdet - INFO - Epoch [2][250/604]	lr: 1.000e-02, eta: 0:00:48, time: 0.132, data_time: 0.003, 

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 294/294, 131.1 task/s, elapsed: 2s, ETA:     0s

2022-11-08 03:14:55,239 - mmdet - INFO - Epoch(val) [2][294]	mAP: 0.7610


# 验证跟踪模型（检测器+跟踪）

In [49]:
import mmcv
from mmdet.apis import set_random_seed
cfg = mmcv.Config.fromfile('./configs/mot/deepsort/deepsort_faster-rcnn_fpn_4e_visdrone_mot.py')
cfg.data_root = 'data/VisDrone_MOT_tiny/'
cfg.data.test.ann_file = cfg.data.test.ann_file.replace('data/VisDrone_MOT/','data/VisDrone_MOT_tiny/')
cfg.data.train.ann_file = cfg.data.train.ann_file.replace('data/VisDrone_MOT/','data/VisDrone_MOT_tiny/')
cfg.data.val.ann_file = cfg.data.val.ann_file.replace('data/VisDrone_MOT/','data/VisDrone_MOT_tiny/')

cfg.data.test.img_prefix = cfg.data.test.img_prefix.replace('data/VisDrone_MOT/','data/VisDrone_MOT_tiny/')
cfg.data.train.img_prefix = cfg.data.train.img_prefix.replace('data/VisDrone_MOT/','data/VisDrone_MOT_tiny/')
cfg.data.val.img_prefix = cfg.data.val.img_prefix.replace('data/VisDrone_MOT/','data/VisDrone_MOT_tiny/')

cfg.model.detector.init_cfg.checkpoint = './tutorial_exps/detector/epoch_4.pth'
cfg.model.reid.init_cfg.checkpoint = './tutorial_exps/reid/epoch_2.pth'

cfg.work_dir = './tutorial_exps'
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.device = 'cuda'
# cfg.gpu_ids = range(1)
cfg.gpu_ids = [0]
cfg.data.test.test_mode = True
print(f'Config:\n{cfg.pretty_text}')

Config:
model = dict(
    detector=dict(
        type='FasterRCNN',
        backbone=dict(
            type='ResNet',
            depth=50,
            num_stages=4,
            out_indices=(0, 1, 2, 3),
            frozen_stages=1,
            norm_cfg=dict(type='BN', requires_grad=True),
            norm_eval=True,
            style='pytorch',
            init_cfg=dict(
                type='Pretrained', checkpoint='torchvision://resnet50')),
        neck=dict(
            type='FPN',
            in_channels=[256, 512, 1024, 2048],
            out_channels=256,
            num_outs=5),
        rpn_head=dict(
            type='RPNHead',
            in_channels=256,
            feat_channels=256,
            anchor_generator=dict(
                type='AnchorGenerator',
                scales=[8],
                ratios=[0.5, 1.0, 2.0],
                strides=[4, 8, 16, 32, 64]),
            bbox_coder=dict(
                type='DeltaXYWHBBoxCoder',
                target_means=[0.0,

In [50]:
from mmtrack.datasets import build_dataloader
from mmtrack.apis import init_model
from mmcv.parallel import MMDataParallel
from mmtrack.apis import single_gpu_test
from mmtrack.datasets import build_dataset

dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
    dataset,
    samples_per_gpu=1,
    workers_per_gpu=cfg.data.workers_per_gpu,
    dist=False,
    shuffle=False)

# build the model and load checkpoint
model = init_model(cfg)

model = MMDataParallel(model, device_ids=cfg.gpu_ids)
outputs = single_gpu_test(model, data_loader)

eval_kwargs = cfg.get('evaluation', {}).copy()
# hard-code way to remove EvalHook args
eval_hook_args = [
    'interval', 'tmpdir', 'start', 'gpu_collect', 'save_best',
    'rule', 'by_epoch'
]
for key in eval_hook_args:
    eval_kwargs.pop(key, None)
eval_kwargs.update(dict(metric=['track']))
metric = dataset.evaluate(outputs, **eval_kwargs)
print(metric)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


2022-11-08 04:45:23,709 - mmcv - INFO - initialize FasterRCNN with init_cfg {'type': 'Pretrained', 'checkpoint': './tutorial_exps/detector/epoch_4.pth'}
2022-11-08 04:45:23,710 - mmcv - INFO - load model from: ./tutorial_exps/detector/epoch_4.pth
2022-11-08 04:45:23,710 - mmcv - INFO - load checkpoint from local path: ./tutorial_exps/detector/epoch_4.pth
2022-11-08 04:45:23,911 - mmcv - INFO - initialize BaseReID with init_cfg {'type': 'Pretrained', 'checkpoint': './tutorial_exps/reid/epoch_2.pth'}
2022-11-08 04:45:23,911 - mmcv - INFO - load model from: ./tutorial_exps/reid/epoch_2.pth
2022-11-08 04:45:23,912 - mmcv - INFO - load checkpoint from local path: ./tutorial_exps/reid/epoch_2.pth
2022-11-08 04:45:24,029 - mmcv - INFO - 
detector.backbone.conv1.weight - torch.Size([64, 3, 7, 7]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 04:45:24,029 - mmcv - INFO - 
detector.backbone.bn1.weight - torch.Size([64]): 
PretrainedInit: load from ./tutorial_exps

2022-11-08 04:45:24,049 - mmcv - INFO - 
detector.backbone.layer2.0.downsample.1.bias - torch.Size([512]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 04:45:24,049 - mmcv - INFO - 
detector.backbone.layer2.1.conv1.weight - torch.Size([128, 512, 1, 1]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 04:45:24,049 - mmcv - INFO - 
detector.backbone.layer2.1.bn1.weight - torch.Size([128]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 04:45:24,050 - mmcv - INFO - 
detector.backbone.layer2.1.bn1.bias - torch.Size([128]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 04:45:24,050 - mmcv - INFO - 
detector.backbone.layer2.1.conv2.weight - torch.Size([128, 128, 3, 3]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 04:45:24,050 - mmcv - INFO - 
detector.backbone.layer2.1.bn2.weight - torch.Size([128]): 
PretrainedInit: load from ./tutor

2022-11-08 04:45:24,066 - mmcv - INFO - 
detector.backbone.layer3.1.bn3.bias - torch.Size([1024]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 04:45:24,067 - mmcv - INFO - 
detector.backbone.layer3.2.conv1.weight - torch.Size([256, 1024, 1, 1]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 04:45:24,067 - mmcv - INFO - 
detector.backbone.layer3.2.bn1.weight - torch.Size([256]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 04:45:24,067 - mmcv - INFO - 
detector.backbone.layer3.2.bn1.bias - torch.Size([256]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 04:45:24,068 - mmcv - INFO - 
detector.backbone.layer3.2.conv2.weight - torch.Size([256, 256, 3, 3]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 04:45:24,068 - mmcv - INFO - 
detector.backbone.layer3.2.bn2.weight - torch.Size([256]): 
PretrainedInit: load from ./tutorial_exp

2022-11-08 04:45:24,084 - mmcv - INFO - 
detector.backbone.layer4.0.downsample.1.bias - torch.Size([2048]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 04:45:24,084 - mmcv - INFO - 
detector.backbone.layer4.1.conv1.weight - torch.Size([512, 2048, 1, 1]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 04:45:24,085 - mmcv - INFO - 
detector.backbone.layer4.1.bn1.weight - torch.Size([512]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 04:45:24,085 - mmcv - INFO - 
detector.backbone.layer4.1.bn1.bias - torch.Size([512]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 04:45:24,085 - mmcv - INFO - 
detector.backbone.layer4.1.conv2.weight - torch.Size([512, 512, 3, 3]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 04:45:24,086 - mmcv - INFO - 
detector.backbone.layer4.1.bn2.weight - torch.Size([512]): 
PretrainedInit: load from ./tut

2022-11-08 04:45:24,101 - mmcv - INFO - 
detector.roi_head.bbox_head.shared_fcs.1.bias - torch.Size([1024]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 04:45:24,102 - mmcv - INFO - 
reid.backbone.conv1.weight - torch.Size([64, 3, 7, 7]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 04:45:24,102 - mmcv - INFO - 
reid.backbone.bn1.weight - torch.Size([64]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 04:45:24,103 - mmcv - INFO - 
reid.backbone.bn1.bias - torch.Size([64]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 04:45:24,103 - mmcv - INFO - 
reid.backbone.layer1.0.conv1.weight - torch.Size([64, 64, 1, 1]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 04:45:24,103 - mmcv - INFO - 
reid.backbone.layer1.0.bn1.weight - torch.Size([64]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 04:45:24,104 - mmcv - INFO - 

2022-11-08 04:45:24,120 - mmcv - INFO - 
reid.backbone.layer2.1.bn2.bias - torch.Size([128]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 04:45:24,120 - mmcv - INFO - 
reid.backbone.layer2.1.conv3.weight - torch.Size([512, 128, 1, 1]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 04:45:24,121 - mmcv - INFO - 
reid.backbone.layer2.1.bn3.weight - torch.Size([512]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 04:45:24,121 - mmcv - INFO - 
reid.backbone.layer2.1.bn3.bias - torch.Size([512]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 04:45:24,122 - mmcv - INFO - 
reid.backbone.layer2.2.conv1.weight - torch.Size([128, 512, 1, 1]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 04:45:24,122 - mmcv - INFO - 
reid.backbone.layer2.2.bn1.weight - torch.Size([128]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 04:45:24,122 

2022-11-08 04:45:24,139 - mmcv - INFO - 
reid.backbone.layer3.2.bn3.bias - torch.Size([1024]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 04:45:24,139 - mmcv - INFO - 
reid.backbone.layer3.3.conv1.weight - torch.Size([256, 1024, 1, 1]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 04:45:24,140 - mmcv - INFO - 
reid.backbone.layer3.3.bn1.weight - torch.Size([256]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 04:45:24,140 - mmcv - INFO - 
reid.backbone.layer3.3.bn1.bias - torch.Size([256]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 04:45:24,140 - mmcv - INFO - 
reid.backbone.layer3.3.conv2.weight - torch.Size([256, 256, 3, 3]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 04:45:24,141 - mmcv - INFO - 
reid.backbone.layer3.3.bn2.weight - torch.Size([256]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 04:45:24,14

2022-11-08 04:45:24,157 - mmcv - INFO - 
reid.backbone.layer4.2.bn1.bias - torch.Size([512]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 04:45:24,158 - mmcv - INFO - 
reid.backbone.layer4.2.conv2.weight - torch.Size([512, 512, 3, 3]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 04:45:24,158 - mmcv - INFO - 
reid.backbone.layer4.2.bn2.weight - torch.Size([512]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 04:45:24,159 - mmcv - INFO - 
reid.backbone.layer4.2.bn2.bias - torch.Size([512]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 04:45:24,159 - mmcv - INFO - 
reid.backbone.layer4.2.conv3.weight - torch.Size([2048, 512, 1, 1]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 04:45:24,159 - mmcv - INFO - 
reid.backbone.layer4.2.bn3.weight - torch.Size([2048]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 04:45:24,16

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 118/118, 14.2 task/s, elapsed: 8s, ETA:     0s---CLEAR MOT Evaluation---
Accumulating...
Evaluating...
Rendering...

            IDF1    MOTA  MOTP   FP  FN IDSw Rcll Prcn MT PT ML  FM
pedestrian  2.6% -166.6% 0.668 1017 568    9 5.0% 2.9%  1  0 11   4
people       NaN     NaN   NaN    0   0    0  NaN  NaN  0  0  0   0
OVERALL     2.6% -166.6% 0.668 1017 568    9 5.0% 2.9%  1  0 11   4
AVERAGE     1.3%  -83.3% 0.334 1017 568    9 2.5% 1.4%  1  0 11   4
Evaluation finishes with 0.26 s.
{'IDF1': 0.026, 'MOTA': -1.666, 'MOTP': 0.668, 'FP': 1017, 'FN': 568, 'IDSw': 9, 'Rcll': 0.05, 'Prcn': 0.029, 'MT': 1, 'PT': 0, 'ML': 11, 'FM': 4, 'track_OVERALL_copypaste': '0.026 -1.666 0.668 1017 568 9 0.050 0.029 1 0 11 4 ', 'track_AVERAGE_copypaste': '0.013 -0.833 0.334 1017 568 9 0.025 0.014 1 0 11 4 '}


# 推理（检测+跟踪）

In [53]:
# run mot demo
import mmcv
import tempfile
from mmtrack.apis import inference_mot, init_model
# mot_config = './configs/mot/deepsort/deepsort_faster-rcnn_fpn_4e_visdrone_mot.py'
cfg = mmcv.Config.fromfile('./configs/mot/deepsort/deepsort_faster-rcnn_fpn_4e_visdrone_mot.py')
cfg.model.detector.init_cfg.checkpoint = './tutorial_exps/detector/epoch_4.pth'
cfg.model.reid.init_cfg.checkpoint = './tutorial_exps/reid/epoch_2.pth'
input_video = './demo/visdrone_demo.mp4'
imgs = mmcv.VideoReader(input_video)
# build the model from a config file
# mot_model = init_model(mot_config, device='cuda:0')
mot_model = init_model(cfg)
prog_bar = mmcv.ProgressBar(len(imgs))
out_dir = tempfile.TemporaryDirectory()
out_path = out_dir.name
# test and show/save the images
for i, img in enumerate(imgs):
    result = inference_mot(mot_model, img, frame_id=i)
    mot_model.show_result(
            img,
            result,
            show=False,
            wait_time=int(1000. / imgs.fps),
            out_file=f'{out_path}/{i:06d}.jpg')
    prog_bar.update()

output = './demo/visdrone_demo_result.mp4'
print(f'\n making the output video at {output} with a FPS of {imgs.fps}')
mmcv.frames2video(out_path, output, fps=imgs.fps, fourcc='mp4v')
out_dir.cleanup()

2022-11-08 04:49:29,758 - mmcv - INFO - initialize FasterRCNN with init_cfg {'type': 'Pretrained', 'checkpoint': './tutorial_exps/detector/epoch_4.pth'}
2022-11-08 04:49:29,758 - mmcv - INFO - load model from: ./tutorial_exps/detector/epoch_4.pth
2022-11-08 04:49:29,759 - mmcv - INFO - load checkpoint from local path: ./tutorial_exps/detector/epoch_4.pth
2022-11-08 04:49:29,963 - mmcv - INFO - initialize BaseReID with init_cfg {'type': 'Pretrained', 'checkpoint': './tutorial_exps/reid/epoch_2.pth'}
2022-11-08 04:49:29,963 - mmcv - INFO - load model from: ./tutorial_exps/reid/epoch_2.pth
2022-11-08 04:49:29,964 - mmcv - INFO - load checkpoint from local path: ./tutorial_exps/reid/epoch_2.pth
2022-11-08 04:49:30,093 - mmcv - INFO - 
detector.backbone.conv1.weight - torch.Size([64, 3, 7, 7]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 04:49:30,094 - mmcv - INFO - 
detector.backbone.bn1.weight - torch.Size([64]): 
PretrainedInit: load from ./tutorial_exps

2022-11-08 04:49:30,113 - mmcv - INFO - 
detector.backbone.layer2.0.downsample.1.bias - torch.Size([512]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 04:49:30,114 - mmcv - INFO - 
detector.backbone.layer2.1.conv1.weight - torch.Size([128, 512, 1, 1]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 04:49:30,114 - mmcv - INFO - 
detector.backbone.layer2.1.bn1.weight - torch.Size([128]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 04:49:30,115 - mmcv - INFO - 
detector.backbone.layer2.1.bn1.bias - torch.Size([128]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 04:49:30,115 - mmcv - INFO - 
detector.backbone.layer2.1.conv2.weight - torch.Size([128, 128, 3, 3]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 04:49:30,116 - mmcv - INFO - 
detector.backbone.layer2.1.bn2.weight - torch.Size([128]): 
PretrainedInit: load from ./tutor

2022-11-08 04:49:30,136 - mmcv - INFO - 
detector.backbone.layer3.1.bn3.bias - torch.Size([1024]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 04:49:30,136 - mmcv - INFO - 
detector.backbone.layer3.2.conv1.weight - torch.Size([256, 1024, 1, 1]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 04:49:30,137 - mmcv - INFO - 
detector.backbone.layer3.2.bn1.weight - torch.Size([256]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 04:49:30,137 - mmcv - INFO - 
detector.backbone.layer3.2.bn1.bias - torch.Size([256]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 04:49:30,138 - mmcv - INFO - 
detector.backbone.layer3.2.conv2.weight - torch.Size([256, 256, 3, 3]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 04:49:30,138 - mmcv - INFO - 
detector.backbone.layer3.2.bn2.weight - torch.Size([256]): 
PretrainedInit: load from ./tutorial_exp

2022-11-08 04:49:30,158 - mmcv - INFO - 
detector.backbone.layer4.0.downsample.1.bias - torch.Size([2048]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 04:49:30,158 - mmcv - INFO - 
detector.backbone.layer4.1.conv1.weight - torch.Size([512, 2048, 1, 1]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 04:49:30,159 - mmcv - INFO - 
detector.backbone.layer4.1.bn1.weight - torch.Size([512]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 04:49:30,159 - mmcv - INFO - 
detector.backbone.layer4.1.bn1.bias - torch.Size([512]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 04:49:30,159 - mmcv - INFO - 
detector.backbone.layer4.1.conv2.weight - torch.Size([512, 512, 3, 3]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 04:49:30,160 - mmcv - INFO - 
detector.backbone.layer4.1.bn2.weight - torch.Size([512]): 
PretrainedInit: load from ./tut

2022-11-08 04:49:30,180 - mmcv - INFO - 
detector.roi_head.bbox_head.shared_fcs.1.bias - torch.Size([1024]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 04:49:30,181 - mmcv - INFO - 
reid.backbone.conv1.weight - torch.Size([64, 3, 7, 7]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 04:49:30,181 - mmcv - INFO - 
reid.backbone.bn1.weight - torch.Size([64]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 04:49:30,182 - mmcv - INFO - 
reid.backbone.bn1.bias - torch.Size([64]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 04:49:30,182 - mmcv - INFO - 
reid.backbone.layer1.0.conv1.weight - torch.Size([64, 64, 1, 1]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 04:49:30,183 - mmcv - INFO - 
reid.backbone.layer1.0.bn1.weight - torch.Size([64]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 04:49:30,183 - mmcv - INFO - 

2022-11-08 04:49:30,205 - mmcv - INFO - 
reid.backbone.layer2.1.bn2.bias - torch.Size([128]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 04:49:30,205 - mmcv - INFO - 
reid.backbone.layer2.1.conv3.weight - torch.Size([512, 128, 1, 1]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 04:49:30,205 - mmcv - INFO - 
reid.backbone.layer2.1.bn3.weight - torch.Size([512]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 04:49:30,206 - mmcv - INFO - 
reid.backbone.layer2.1.bn3.bias - torch.Size([512]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 04:49:30,206 - mmcv - INFO - 
reid.backbone.layer2.2.conv1.weight - torch.Size([128, 512, 1, 1]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 04:49:30,207 - mmcv - INFO - 
reid.backbone.layer2.2.bn1.weight - torch.Size([128]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 04:49:30,207 

2022-11-08 04:49:30,228 - mmcv - INFO - 
reid.backbone.layer3.2.bn3.bias - torch.Size([1024]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 04:49:30,229 - mmcv - INFO - 
reid.backbone.layer3.3.conv1.weight - torch.Size([256, 1024, 1, 1]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 04:49:30,229 - mmcv - INFO - 
reid.backbone.layer3.3.bn1.weight - torch.Size([256]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 04:49:30,230 - mmcv - INFO - 
reid.backbone.layer3.3.bn1.bias - torch.Size([256]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 04:49:30,230 - mmcv - INFO - 
reid.backbone.layer3.3.conv2.weight - torch.Size([256, 256, 3, 3]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 04:49:30,231 - mmcv - INFO - 
reid.backbone.layer3.3.bn2.weight - torch.Size([256]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 04:49:30,23

2022-11-08 04:49:30,252 - mmcv - INFO - 
reid.backbone.layer4.2.bn1.bias - torch.Size([512]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 04:49:30,252 - mmcv - INFO - 
reid.backbone.layer4.2.conv2.weight - torch.Size([512, 512, 3, 3]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 04:49:30,253 - mmcv - INFO - 
reid.backbone.layer4.2.bn2.weight - torch.Size([512]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 04:49:30,253 - mmcv - INFO - 
reid.backbone.layer4.2.bn2.bias - torch.Size([512]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 04:49:30,254 - mmcv - INFO - 
reid.backbone.layer4.2.conv3.weight - torch.Size([2048, 512, 1, 1]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 04:49:30,254 - mmcv - INFO - 
reid.backbone.layer4.2.bn3.weight - torch.Size([2048]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 04:49:30,25

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 109/109, 10.5 task/s, elapsed: 10s, ETA:     0s
 making the output video at ./demo/visdrone_demo_result.mp4 with a FPS of 10.0
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 109/109, 66.6 task/s, elapsed: 2s, ETA:     0s
